In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn;
import datetime as dt;
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")


In [3]:
test=pd.read_csv('test.csv')
test.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
#Id column has no significance, so lets drop it
test.drop('Id',axis =1,inplace= True);

test.head(1)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal


In [5]:

for i in test.columns:
    if test[i].isnull().sum() >1:
        print('nulls in '+i+ ' = ' + '{} %'.format( np.round ( test[i].isnull().sum()*100 / len(test),2) ) )

nulls in MSZoning = 0.27 %
nulls in LotFrontage = 15.56 %
nulls in Alley = 92.67 %
nulls in Utilities = 0.14 %
nulls in MasVnrType = 1.1 %
nulls in MasVnrArea = 1.03 %
nulls in BsmtQual = 3.02 %
nulls in BsmtCond = 3.08 %
nulls in BsmtExposure = 3.02 %
nulls in BsmtFinType1 = 2.88 %
nulls in BsmtFinType2 = 2.88 %
nulls in BsmtFullBath = 0.14 %
nulls in BsmtHalfBath = 0.14 %
nulls in Functional = 0.14 %
nulls in FireplaceQu = 50.03 %
nulls in GarageType = 5.21 %
nulls in GarageYrBlt = 5.35 %
nulls in GarageFinish = 5.35 %
nulls in GarageQual = 5.35 %
nulls in GarageCond = 5.35 %
nulls in PoolQC = 99.79 %
nulls in Fence = 80.12 %
nulls in MiscFeature = 96.5 %


In [6]:
test.drop(['Alley','PoolQC','Fence','MiscFeature'],axis=1,inplace=True)

In [7]:
# replacing remaining (categorical) values with mode

test.bfill(inplace = True)
test.ffill(inplace = True)
    
# total_null_values after replacement   
print( test.isnull().sum().sum() )

0


In [8]:
year_features = ['YearRemodAdd', 'YearBuilt', 'YrSold', 'GarageYrBlt']

test['date'] = pd.to_datetime('today').date()
test['curr_year'] = pd.DatetimeIndex(test['date']).year

for i in year_features:
    test[i] = test['curr_year'] - test[i]

test[['YearRemodAdd', 'YearBuilt', 'YrSold', 'GarageYrBlt']].head()

,YearRemodAdd,YearBuilt,YrSold,GarageYrBlt
0,60,60,11,60.0
1,63,63,11,63.0
2,23,24,11,24.0
3,23,23,11,23.0
4,29,29,11,29.0


In [9]:
test.drop(columns=['date','curr_year','YrSold','MoSold'], axis = 1, inplace = True)

In [10]:
# from data descriptio.txt, we grtto know that garage qual and exterqual, bsmntnsf, condition2 can be obtained from other featues, so lets drop them
# but we already removed garage condition so dont remove garage qual.
test.drop(columns=['Condition2','BsmtUnfSF','OverallQual','ExterQual','Exterior2nd'], axis = 1, inplace = True)

In [11]:
#select all numerical variables and copy paste it
test.select_dtypes(exclude=object).columns


Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal'],
      dtype='object')

In [12]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
columns_to_scale = [ 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']

from sklearn.preprocessing import RobustScaler

rs = RobustScaler()
scaled_columns_rs  = rs.fit_transform(test[columns_to_scale])

scaled_df_rs = pd.DataFrame(data=scaled_columns_rs, columns=[ 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal'] )


mm = MinMaxScaler()
scaled_columns_mm  = mm.fit_transform(test[columns_to_scale])

scaled_df_mm = pd.DataFrame(data=scaled_columns_mm, columns=['MSSubClass', 'LotFrontage', 'LotArea', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
       'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal'] )

In [13]:
test_cat = test.select_dtypes(include='object' )
# train_cat.info()
print(test_cat.shape )

for i in test_cat:
    print(i,'=',test_cat[i].nunique(),  test_cat[i].unique() )
test_cat.shape[1]

(1459, 36)
MSZoning = 5 ['RH' 'RL' 'RM' 'FV' 'C (all)']
Street = 2 ['Pave' 'Grvl']
LotShape = 4 ['Reg' 'IR1' 'IR2' 'IR3']
LandContour = 4 ['Lvl' 'HLS' 'Bnk' 'Low']
Utilities = 1 ['AllPub']
LotConfig = 5 ['Inside' 'Corner' 'FR2' 'CulDSac' 'FR3']
LandSlope = 3 ['Gtl' 'Mod' 'Sev']
Neighborhood = 25 ['NAmes' 'Gilbert' 'StoneBr' 'BrDale' 'NPkVill' 'NridgHt' 'Blmngtn'
 'NoRidge' 'Somerst' 'SawyerW' 'Sawyer' 'NWAmes' 'OldTown' 'BrkSide'
 'ClearCr' 'SWISU' 'Edwards' 'CollgCr' 'Crawfor' 'Blueste' 'IDOTRR'
 'Mitchel' 'Timber' 'MeadowV' 'Veenker']
Condition1 = 9 ['Feedr' 'Norm' 'PosN' 'RRNe' 'Artery' 'RRNn' 'PosA' 'RRAn' 'RRAe']
BldgType = 5 ['1Fam' 'TwnhsE' 'Twnhs' 'Duplex' '2fmCon']
HouseStyle = 7 ['1Story' '2Story' 'SLvl' '1.5Fin' 'SFoyer' '2.5Unf' '1.5Unf']
RoofStyle = 6 ['Gable' 'Hip' 'Gambrel' 'Flat' 'Mansard' 'Shed']
RoofMatl = 4 ['CompShg' 'Tar&Grv' 'WdShake' 'WdShngl']
Exterior1st = 13 ['VinylSd' 'Wd Sdng' 'HdBoard' 'Plywood' 'MetalSd' 'CemntBd' 'WdShing'
 'BrkFace' 'AsbShng' 'BrkComm' '

36

In [14]:
# encoding neighborhood values for first 20 most occured values
test_cat.Neighborhood.value_counts().head()

top_20 = [ x for x in test_cat.Neighborhood.value_counts().sort_values(ascending=False).head(20).index ]
print(top_20)
print()


for uq_value in top_20:
    col = 'Neighborhood';
    test_cat[col+'_'+uq_value] = np.where( test_cat['Neighborhood'] == uq_value,1,0 ) 
#to see the new columns created    
#print(train[['Neighborhood']+top_20])
#can dsee all the new values apppended to the list
test_cat
print(test_cat.shape[1])
#print(train_cat.columns)


['NAmes', 'OldTown', 'CollgCr', 'Somerst', 'Edwards', 'NridgHt', 'Gilbert', 'Sawyer', 'SawyerW', 'Mitchel', 'NWAmes', 'IDOTRR', 'Crawfor', 'BrkSide', 'Timber', 'NoRidge', 'StoneBr', 'SWISU', 'MeadowV', 'ClearCr']

56


In [15]:
#Exterior1st
test_cat.Exterior1st.value_counts().head()


top_10 = [ x for x in test_cat.Exterior1st.value_counts().sort_values(ascending=False).head(10).index ]
print(top_10)
print()


for uq_value in top_10:
    col = 'Exterior1st';
    test_cat[col+'_'+ uq_value] = np.where( test_cat['Exterior1st'] == uq_value,1,0) 
# to see the new columns created    
# print(train[['Exterior1st']+top_10])

#can dsee all the new values apppended to the list
test_cat.shape[1]

['VinylSd', 'MetalSd', 'HdBoard', 'Wd Sdng', 'Plywood', 'CemntBd', 'BrkFace', 'WdShing', 'AsbShng', 'Stucco']



66

In [16]:
#RoofMatl
print( test_cat.RoofMatl.value_counts() )
# u can notice last 4 categories have only one value so lets choose only top 5

top_5 = [ x for x in test_cat.RoofMatl.value_counts().sort_values(ascending=False).head(4).index ]
print(top_10)
print()


for uq_value in top_5:
    col = 'RoofMatl';
    test_cat[col+'_'+ uq_value] = np.where(test_cat['RoofMatl'] == uq_value,1,0) 
    
# to see the new columns created    
# print(train[['RoofMatl']+top_10])

#can dsee all the new values apppended to the list
test_cat.shape[1]

CompShg    1442
Tar&Grv      12
WdShake       4
WdShngl       1
Name: RoofMatl, dtype: int64
['VinylSd', 'MetalSd', 'HdBoard', 'Wd Sdng', 'Plywood', 'CemntBd', 'BrkFace', 'WdShing', 'AsbShng', 'Stucco']



70

In [17]:
#now lets drop those 3 columns
test_cat.drop(columns= ['RoofMatl','Exterior1st','Neighborhood'], axis  = 1, inplace = True)

In [18]:

test_cat = pd.get_dummies(test_cat,drop_first=True)
test_cat.columns.to_list()

['Neighborhood_NAmes',
 'Neighborhood_OldTown',
 'Neighborhood_CollgCr',
 'Neighborhood_Somerst',
 'Neighborhood_Edwards',
 'Neighborhood_NridgHt',
 'Neighborhood_Gilbert',
 'Neighborhood_Sawyer',
 'Neighborhood_SawyerW',
 'Neighborhood_Mitchel',
 'Neighborhood_NWAmes',
 'Neighborhood_IDOTRR',
 'Neighborhood_Crawfor',
 'Neighborhood_BrkSide',
 'Neighborhood_Timber',
 'Neighborhood_NoRidge',
 'Neighborhood_StoneBr',
 'Neighborhood_SWISU',
 'Neighborhood_MeadowV',
 'Neighborhood_ClearCr',
 'Exterior1st_VinylSd',
 'Exterior1st_MetalSd',
 'Exterior1st_HdBoard',
 'Exterior1st_Wd Sdng',
 'Exterior1st_Plywood',
 'Exterior1st_CemntBd',
 'Exterior1st_BrkFace',
 'Exterior1st_WdShing',
 'Exterior1st_AsbShng',
 'Exterior1st_Stucco',
 'RoofMatl_CompShg',
 'RoofMatl_Tar&Grv',
 'RoofMatl_WdShake',
 'RoofMatl_WdShngl',
 'MSZoning_FV',
 'MSZoning_RH',
 'MSZoning_RL',
 'MSZoning_RM',
 'Street_Pave',
 'LotShape_IR2',
 'LotShape_IR3',
 'LotShape_Reg',
 'LandContour_HLS',
 'LandContour_Low',
 'LandContour_

In [19]:
# concat two dataframes train_cat and scaled_column_df 
print(test_cat.shape)
type(test_cat)
type(scaled_df_rs)
type(scaled_df_mm)
print(scaled_df_rs.columns.to_list())
print(test_cat.columns.to_list())

(1459, 158)
['MSSubClass', 'LotFrontage', 'LotArea', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']
['Neighborhood_NAmes', 'Neighborhood_OldTown', 'Neighborhood_CollgCr', 'Neighborhood_Somerst', 'Neighborhood_Edwards', 'Neighborhood_NridgHt', 'Neighborhood_Gilbert', 'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Mitchel', 'Neighborhood_NWAmes', 'Neighborhood_IDOTRR', 'Neighborhood_Crawfor', 'Neighborhood_BrkSide', 'Neighborhood_Timber', 'Neighborhood_NoRidge', 'Neighborhood_StoneBr', 'Neighborhood_SWISU', 'Neighborhood_MeadowV', 'Neighborhood_ClearCr', 'Exterior1st_VinylSd', 'Exterior1st_MetalSd', 'Exterior1st_HdB

In [20]:

X_test_rs =np.concatenate( [scaled_df_rs, test_cat],axis = 1)

X_testd_rsd = pd.DataFrame( X_test_rs, columns = 
                           ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal','Neighborhood_NAmes', 'Neighborhood_OldTown', 'Neighborhood_CollgCr', 'Neighborhood_Somerst', 'Neighborhood_Edwards', 'Neighborhood_NridgHt', 'Neighborhood_Gilbert', 'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Mitchel', 'Neighborhood_NWAmes', 'Neighborhood_IDOTRR', 'Neighborhood_Crawfor', 'Neighborhood_BrkSide', 'Neighborhood_Timber', 'Neighborhood_NoRidge', 'Neighborhood_StoneBr', 'Neighborhood_SWISU', 'Neighborhood_MeadowV', 'Neighborhood_ClearCr', 'Exterior1st_VinylSd', 'Exterior1st_MetalSd', 'Exterior1st_HdBoard', 'Exterior1st_Wd Sdng', 'Exterior1st_Plywood', 'Exterior1st_CemntBd', 'Exterior1st_BrkFace', 'Exterior1st_WdShing', 'Exterior1st_AsbShng', 'Exterior1st_Stucco', 'RoofMatl_CompShg', 'RoofMatl_Tar&Grv', 'RoofMatl_WdShake', 'RoofMatl_WdShngl', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Pave', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Mod', 'LandSlope_Sev', 'Condition1_Feedr', 'Condition1_Norm', 'Condition1_PosA', 'Condition1_PosN', 'Condition1_RRAe', 'Condition1_RRAn', 'Condition1_RRNe', 'Condition1_RRNn', 'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs', 'BldgType_TwnhsE', 'HouseStyle_1.5Unf', 'HouseStyle_1Story', 'HouseStyle_2.5Unf', 'HouseStyle_2Story', 'HouseStyle_SFoyer', 'HouseStyle_SLvl', 'RoofStyle_Gable', 'RoofStyle_Gambrel', 'RoofStyle_Hip', 'RoofStyle_Mansard', 'RoofStyle_Shed', 'MasVnrType_BrkFace', 'MasVnrType_None', 'MasVnrType_Stone', 'ExterCond_Fa', 'ExterCond_Gd', 'ExterCond_Po', 'ExterCond_TA', 'Foundation_CBlock', 'Foundation_PConc', 'Foundation_Slab', 'Foundation_Stone', 'Foundation_Wood', 'BsmtQual_Fa', 'BsmtQual_Gd', 'BsmtQual_TA', 'BsmtCond_Gd', 'BsmtCond_Po', 'BsmtCond_TA', 'BsmtExposure_Gd', 'BsmtExposure_Mn', 'BsmtExposure_No', 'BsmtFinType1_BLQ', 'BsmtFinType1_GLQ', 'BsmtFinType1_LwQ', 'BsmtFinType1_Rec', 'BsmtFinType1_Unf', 'BsmtFinType2_BLQ', 'BsmtFinType2_GLQ', 'BsmtFinType2_LwQ', 'BsmtFinType2_Rec', 'BsmtFinType2_Unf', 'Heating_GasW', 'Heating_Grav', 'Heating_Wall', 'HeatingQC_Fa', 'HeatingQC_Gd', 'HeatingQC_Po', 'HeatingQC_TA', 'CentralAir_Y', 'Electrical_FuseF', 'Electrical_FuseP', 'Electrical_SBrkr', 'KitchenQual_Fa', 'KitchenQual_Gd', 'KitchenQual_TA', 'Functional_Maj2', 'Functional_Min1', 'Functional_Min2', 'Functional_Mod', 'Functional_Sev', 'Functional_Typ', 'FireplaceQu_Fa', 'FireplaceQu_Gd', 'FireplaceQu_Po', 'FireplaceQu_TA', 'GarageType_Attchd', 'GarageType_Basment', 'GarageType_BuiltIn', 'GarageType_CarPort', 'GarageType_Detchd', 'GarageFinish_RFn', 'GarageFinish_Unf', 'GarageQual_Gd', 'GarageQual_Po', 'GarageQual_TA', 'GarageCond_Fa', 'GarageCond_Gd', 'GarageCond_Po', 'GarageCond_TA', 'PavedDrive_P', 'PavedDrive_Y', 'SaleType_CWD', 'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD', 'SaleCondition_AdjLand', 'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal', 'SaleCondition_Partial'] )
print( X_testd_rsd.shape )
X_testd_rsd.head()

(1459, 190)


,MSSubClass,LotFrontage,LotArea,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,TotalBsmtSF,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-0.6,0.545455,0.538713,1.0,0.250000,0.756098,0.000000,0.156915,144.0,-0.203846,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,-0.6,0.590909,1.179692,1.0,0.312500,0.829268,0.664615,0.761968,0.0,0.655769,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.2,0.272727,1.073791,0.0,-0.500000,-0.146341,0.000000,0.586436,0.0,-0.115385,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.2,0.454545,0.140313,1.0,-0.520833,-0.146341,0.123077,0.335106,0.0,-0.119231,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,1.4,-1.136364,-1.064825,0.0,-0.395833,0.000000,0.000000,-0.115691,0.0,0.561538,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [21]:

X_test_mm =np.concatenate( [scaled_df_mm, test_cat],axis = 1)

X_testd_mmd = pd.DataFrame( X_test_mm, columns = 
                           ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal','Neighborhood_NAmes', 'Neighborhood_OldTown', 'Neighborhood_CollgCr', 'Neighborhood_Somerst', 'Neighborhood_Edwards', 'Neighborhood_NridgHt', 'Neighborhood_Gilbert', 'Neighborhood_Sawyer', 'Neighborhood_SawyerW', 'Neighborhood_Mitchel', 'Neighborhood_NWAmes', 'Neighborhood_IDOTRR', 'Neighborhood_Crawfor', 'Neighborhood_BrkSide', 'Neighborhood_Timber', 'Neighborhood_NoRidge', 'Neighborhood_StoneBr', 'Neighborhood_SWISU', 'Neighborhood_MeadowV', 'Neighborhood_ClearCr', 'Exterior1st_VinylSd', 'Exterior1st_MetalSd', 'Exterior1st_HdBoard', 'Exterior1st_Wd Sdng', 'Exterior1st_Plywood', 'Exterior1st_CemntBd', 'Exterior1st_BrkFace', 'Exterior1st_WdShing', 'Exterior1st_AsbShng', 'Exterior1st_Stucco', 'RoofMatl_CompShg', 'RoofMatl_Tar&Grv', 'RoofMatl_WdShake', 'RoofMatl_WdShngl', 'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM', 'Street_Pave', 'LotShape_IR2', 'LotShape_IR3', 'LotShape_Reg', 'LandContour_HLS', 'LandContour_Low', 'LandContour_Lvl', 'LotConfig_CulDSac', 'LotConfig_FR2', 'LotConfig_FR3', 'LotConfig_Inside', 'LandSlope_Mod', 'LandSlope_Sev', 'Condition1_Feedr', 'Condition1_Norm', 'Condition1_PosA', 'Condition1_PosN', 'Condition1_RRAe', 'Condition1_RRAn', 'Condition1_RRNe', 'Condition1_RRNn', 'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs', 'BldgType_TwnhsE', 'HouseStyle_1.5Unf', 'HouseStyle_1Story', 'HouseStyle_2.5Unf', 'HouseStyle_2Story', 'HouseStyle_SFoyer', 'HouseStyle_SLvl', 'RoofStyle_Gable', 'RoofStyle_Gambrel', 'RoofStyle_Hip', 'RoofStyle_Mansard', 'RoofStyle_Shed', 'MasVnrType_BrkFace', 'MasVnrType_None', 'MasVnrType_Stone', 'ExterCond_Fa', 'ExterCond_Gd', 'ExterCond_Po', 'ExterCond_TA', 'Foundation_CBlock', 'Foundation_PConc', 'Foundation_Slab', 'Foundation_Stone', 'Foundation_Wood', 'BsmtQual_Fa', 'BsmtQual_Gd', 'BsmtQual_TA', 'BsmtCond_Gd', 'BsmtCond_Po', 'BsmtCond_TA', 'BsmtExposure_Gd', 'BsmtExposure_Mn', 'BsmtExposure_No', 'BsmtFinType1_BLQ', 'BsmtFinType1_GLQ', 'BsmtFinType1_LwQ', 'BsmtFinType1_Rec', 'BsmtFinType1_Unf', 'BsmtFinType2_BLQ', 'BsmtFinType2_GLQ', 'BsmtFinType2_LwQ', 'BsmtFinType2_Rec', 'BsmtFinType2_Unf', 'Heating_GasW', 'Heating_Grav', 'Heating_Wall', 'HeatingQC_Fa', 'HeatingQC_Gd', 'HeatingQC_Po', 'HeatingQC_TA', 'CentralAir_Y', 'Electrical_FuseF', 'Electrical_FuseP', 'Electrical_SBrkr', 'KitchenQual_Fa', 'KitchenQual_Gd', 'KitchenQual_TA', 'Functional_Maj2', 'Functional_Min1', 'Functional_Min2', 'Functional_Mod', 'Functional_Sev', 'Functional_Typ', 'FireplaceQu_Fa', 'FireplaceQu_Gd', 'FireplaceQu_Po', 'FireplaceQu_TA', 'GarageType_Attchd', 'GarageType_Basment', 'GarageType_BuiltIn', 'GarageType_CarPort', 'GarageType_Detchd', 'GarageFinish_RFn', 'GarageFinish_Unf', 'GarageQual_Gd', 'GarageQual_Po', 'GarageQual_TA', 'GarageCond_Fa', 'GarageCond_Gd', 'GarageCond_Po', 'GarageCond_TA', 'PavedDrive_P', 'PavedDrive_Y', 'SaleType_CWD', 'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD', 'SaleCondition_AdjLand', 'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal', 'SaleCondition_Partial'] )
print( X_testd_mmd.shape )
X_testd_mmd.head()

(1459, 190)


,MSSubClass,LotFrontage,LotArea,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,TotalBsmtSF,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.000000,0.329609,0.184147,0.625,0.374046,0.816667,0.000000,0.116708,0.094364,0.173111,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.000000,0.335196,0.232124,0.625,0.396947,0.866667,0.083721,0.230175,0.000000,0.260844,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.235294,0.296089,0.224197,0.500,0.099237,0.200000,0.000000,0.197257,0.000000,0.182139,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.235294,0.318436,0.154326,0.625,0.091603,0.200000,0.015504,0.150125,0.000000,0.181747,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.588235,0.122905,0.064121,0.500,0.137405,0.300000,0.000000,0.065586,0.000000,0.251227,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [22]:
file1 = X_testd_rsd.to_csv('prep_test_rs.csv',index = False)
file2 = X_testd_mmd.to_csv('prep_test_mm.csv',index = False)